[The DOB Glossary](https://www1.nyc.gov/site/buildings/dob/acronym-glossary.page), which may be a handy reference.

[One of several Fontan Architects articles](https://fontanarchitecture.com/building-a-new-home-in-nyc/). This one explains that the architect gets Approval before the Contractor(s) get permits. [This site](https://www.ny-engineers.com/agencies/dob-approval-and-design) describes a similar process. Unfortunately, I don't see approval data on the DOB website, just permit issuance. So I can't tell how long approval took. But I can at least locate new construction.

Another DOB dataset, [Historical Permits 1989-2013](https://data.cityofnewyork.us/Housing-Development/Historical-DOB-Permit-Issuance/bty7-2jhb) but there are older permits in there as well.

In [39]:
import pandas as pd
from geopy.geocoders import Nominatim
import censusgeocode as cg
import datetime

In [3]:
permits = pd.read_csv('../capstone_large_data_sets/DOB_Permit_Issuance.csv')

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_1477/808881869.py:1: DtypeWarning: Columns (8,9,10,15,31,33,36,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  permits = pd.read_csv('../capstone_large_data_sets/DOB_Permit_Issuance.csv')


I downloaded this data on August 2, 2022. It's updated daily,

In [4]:
permits.head()

,BOROUGH,Bin #,House #,Street Name,Job #,Job doc. #,Job Type,Self_Cert,Block,Lot,...,Owner’s House State,Owner’s House Zip Code,Owner's Phone #,DOBRunDate,PERMIT_SI_NO,LATITUDE,LONGITUDE,COUNCIL_DISTRICT,CENSUS_TRACT,NTA_NAME
0,BRONX,2097114,265,EAST 161ST STREET,220411793,2,A2,N,2444.0,1.0,...,NaN,NaN,2123860481.0,05/10/2022 00:00:00,3905783,40.825928,-73.919514,16.0,18302.0,East Concourse-Concourse Village
1,QUEENS,4114282,41-17,MAIN STREET,421692010,2,A2,N,5043.0,11.0,...,NaN,NaN,7183912934.0,05/10/2022 00:00:00,3905784,40.757784,-73.829252,20.0,853.0,Flushing
2,BRONX,2092754,875,TAYLOR AVENUE,220585230,1,A2,N,3637.0,1.0,...,NaN,NaN,2123066936.0,05/10/2022 00:00:00,3905785,40.823072,-73.863399,18.0,42.0,Soundview-Castle Hill-Clason Point-Harding Park
3,BROOKLYN,3185962,161,BAY 35TH ST,340843671,1,A2,Y,6875.0,13.0,...,NaN,NaN,3478738710.0,05/10/2022 00:00:00,3905786,40.596642,-73.993310,47.0,302.0,Bensonhurst East
4,BRONX,2082461,258,CITY ISLAND AVENUE,220657331,1,A2,Y,5642.0,11.0,...,NaN,NaN,2123272277.0,05/10/2022 00:00:00,3905787,40.845730,-73.785907,13.0,516.0,Pelham Bay-Country Club-City Island


In [5]:
permits.shape

(3909190, 60)

There are far more columns than I need. [The NYC Data page for DOB Permits](https://data.cityofnewyork.us/Housing-Development/DOB-Permit-Issuance/ipu4-2q9a) has a list of columns.

For now, I'm going to get rid of most columns but will keep the `job #` to enable reconnecting additional data if it seems helpful later.

I also want to clean up the column names.

In [6]:
permits_backup = permits.copy()

In [214]:
permits = permits_backup.copy()

In [215]:
permits = permits[['BOROUGH', 'Bin #', 'House #', 'Street Name', 'Job #','Job Type', 
                   'Zip Code', 'Bldg Type', 'Residential', 'Work Type', 'Permit Status', 
                   'Filing Status', 'Permit Type', 'Permit Sequence #', 'Filing Date', 'Issuance Date', 'Expiration Date', 'Job Start Date', "Owner's Business Type", 
                   "Non-Profit", 'DOBRunDate', 'PERMIT_SI_NO', 'LATITUDE', 'LONGITUDE', 'COUNCIL_DISTRICT', 'CENSUS_TRACT']]

In [216]:
permits.head()

,BOROUGH,Bin #,House #,Street Name,Job #,Job Type,Zip Code,Bldg Type,Residential,Work Type,Permit Status,Filing Status,Permit Type,Permit Sequence #,Filing Date,Issuance Date,Expiration Date,Job Start Date,Owner's Business Type,Non-Profit,DOBRunDate,PERMIT_SI_NO,LATITUDE,LONGITUDE,COUNCIL_DISTRICT,CENSUS_TRACT
0,BRONX,2097114,265,EAST 161ST STREET,220411793,A2,10451.0,2.0,NaN,MH,ISSUED,RENEWAL,EW,7,05/09/2022,05/09/2022,05/01/2023,05/27/2016,OTHER GOV'T AGENCY,N,05/10/2022 00:00:00,3905783,40.825928,-73.919514,16.0,18302.0
1,QUEENS,4114282,41-17,MAIN STREET,421692010,A2,11355.0,2.0,NaN,MH,ISSUED,RENEWAL,EW,2,05/09/2022,05/09/2022,05/01/2023,10/28/2021,NYC AGENCY,N,05/10/2022 00:00:00,3905784,40.757784,-73.829252,20.0,853.0
2,BRONX,2092754,875,TAYLOR AVENUE,220585230,A2,10473.0,2.0,YES,OT,ISSUED,INITIAL,EW,1,05/09/2022,05/09/2022,11/27/2022,05/09/2022,NYCHA/HHC,N,05/10/2022 00:00:00,3905785,40.823072,-73.863399,18.0,42.0
3,BROOKLYN,3185962,161,BAY 35TH ST,340843671,A2,11214.0,1.0,YES,OT,ISSUED,INITIAL,EW,1,05/09/2022,05/09/2022,04/27/2023,05/09/2022,INDIVIDUAL,N,05/10/2022 00:00:00,3905786,40.596642,-73.993310,47.0,302.0
4,BRONX,2082461,258,CITY ISLAND AVENUE,220657331,A2,10464.0,2.0,NaN,OT,ISSUED,RENEWAL,EW,5,05/09/2022,05/09/2022,05/01/2023,07/23/2018,INDIVIDUAL,N,05/10/2022 00:00:00,3905787,40.845730,-73.785907,13.0,516.0


A [useful explanation of job types](https://fontanarchitecture.com/alt-1-alt-2-nyc/).

[This one](https://ownersrepny.com/2011/10/12/nyc-building-permits-what-does-nb-alteration-type-i-ii-iii-and-directive-14-mean/) has even more detail.

In [217]:
permits.columns = permits.columns.str.replace("'", "").str.replace('#', 'no').str.replace(' ', '_').str.lower()

In [218]:
permits.isna().sum()

borough                       0
bin_no                        0
house_no                      4
street_name                   4
job_no                        0
job_type                      0
zip_code                   1166
bldg_type                 57389
residential             2330686
work_type                698603
permit_status             11219
filing_status                 0
permit_type                   1
permit_sequence_no            0
filing_date                   1
issuance_date             19937
expiration_date           11399
job_start_date               10
owners_business_type     171472
non-profit               167207
dobrundate                    4
permit_si_no                  0
latitude                  11160
longitude                 11160
council_district          11160
census_tract              11160
dtype: int64

Just based on what I see above, I'll explore if 'residential' `NaN` might actually be non-residential. 
I wonder if non-profit will turn out to be the same.

The buildings that are missing latitude, lognitude, council_district, and census_tract all appear to be the same (by virtue of their being the same number, which would be quite a coincidence). I'll explore what these are. It's a minute percentage of the data set, if I wind up needing to drop them.

Zip code may not be necessary to identify the location of these buildings. Though ultimately it will be a question of whether I use longitude, latitude or street addresses.

I need to explore bldg_type, work_type, permit_status, owners_business_type.

It's odd so many issuance and expiration date's are missing. I can use job start date as a proxy though. I can look at time between filing and that as a measure of time it takes to get to work.

house_no, street_name, permit_type, and dobrundate we'll see, but they're so few dropping won't be a problem.

In [219]:
permits['residential'].value_counts(dropna = False)

NaN    2330686
YES    1578504
Name: residential, dtype: int64

In [220]:
permits['residential'].value_counts()[0]/permits.shape[0]

0.4037931131513178

It appears that in the `residential` column, `NaN`s are not residential. I want to look only at residential building, so I'm going to eliminate the non-residential buildings, which will also hopefully help with the clean up of other `NaN`s.

After I do that, I'm going to look at the `job_type` to reduce down to new buildings, which will hopefully reduce the number of `NaN`s further.

In [221]:
permits = permits[permits['residential'] == 'YES']
permits.shape

(1578504, 26)

In [222]:
permits['job_type'].value_counts()

A2    668065
NB    448845
A1    306156
A3    155437
SG         1
Name: job_type, dtype: int64

In [223]:
permits = permits[permits['job_type']=='NB']
permits.shape

(448845, 26)

In [224]:
permits.shape[0]/permits_backup.shape[0]

0.11481790345314502

New residential buildings are 11.5% of the total permits in the database.

examining `permit_type`. This actually (mostly) corresponds to "Work Type Acronyms" on [the DOB acronym glossary page](https://www1.nyc.gov/site/buildings/dob/acronym-glossary.page), not "Permit Type Acronyms" on the page. That said, this [Permit Type and Status Code page](https://www1.nyc.gov/site/buildings/industry/permit-type-and-job-status-codes.page) may have more information... Yup, it does. 
- NB = New Building
- EQ = Equipment
- PL = Plumbing
- FO = Foundation/Earthwork
- EW = Equipment Work

In [225]:
permits['permit_type'].value_counts(dropna = False)

NB    217407
EQ    103526
PL     93959
FO     33935
EW        18
Name: permit_type, dtype: int64

In [226]:
permits.shape

(448845, 26)

In [227]:
permits = permits[permits['permit_type'] == 'NB']

In [228]:
permits.shape

(217407, 26)

In [229]:
permits[(permits['permit_type'] == 'NB') & (permits['filing_status'] != 'RENEWAL')].head(20)

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,bldg_type,residential,work_type,permit_status,filing_status,permit_type,permit_sequence_no,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude,council_district,census_tract
333,QUEENS,4624330,30-11,QUEENS BOULEVARD,402284675,NB,11101.0,2.0,YES,NaN,ISSUED,INITIAL,NB,1,06/26/2008,06/30/2008,11/10/2008,06/30/2008,INDIVIDUAL,N,05/11/2022 00:00:00,2065732,40.748806,-73.937263,26.0,171.0
386,MANHATTAN,1056547,2686,BROADWAY,121207354,NB,10025.0,2.0,YES,NaN,ISSUED,INITIAL,NB,1,05/11/2022,05/11/2022,05/11/2023,05/11/2022,PARTNERSHIP,N,05/12/2022 00:00:00,3906101,40.798817,-73.968740,7.0,191.0
450,MANHATTAN,1813272,645,EAST 9 STREET,123910526,NB,10009.0,2.0,YES,NaN,RE-ISSUED,INITIAL,NB,1,05/11/2022,05/11/2022,05/11/2023,05/11/2022,CORPORATION,N,05/12/2022 00:00:00,3906173,40.725754,-73.979064,2.0,28.0
511,MANHATTAN,1812187,140,HILLSIDE AVENUE,121189524,NB,10040.0,2.0,YES,NaN,ISSUED,INITIAL,NB,1,05/11/2022,05/11/2022,05/11/2023,05/11/2022,CORPORATION,N,05/12/2022 00:00:00,3906240,40.860296,-73.926125,10.0,283.0
516,BRONX,2823631,368,EAST 152 STREET,220586168,NB,10455.0,2.0,YES,NaN,ISSUED,INITIAL,NB,1,05/11/2022,05/11/2022,02/23/2023,05/11/2022,INDIVIDUAL,N,05/12/2022 00:00:00,3906247,40.818565,-73.918118,17.0,67.0
526,BROOKLYN,3429007,3410,FARRAGUT ROAD,321588215,NB,11210.0,2.0,YES,NaN,ISSUED,INITIAL,NB,1,05/11/2022,05/18/2022,04/27/2023,05/18/2022,INDIVIDUAL,N,05/19/2022 00:00:00,3906243,40.636513,-73.943944,45.0,784.0
530,BROOKLYN,3121674,1457,FLATBUSH AVENUE,321827163,NB,11210.0,2.0,YES,NaN,ISSUED,INITIAL,NB,1,05/11/2022,05/11/2022,05/11/2023,05/11/2022,CORPORATION,N,05/12/2022 00:00:00,3906259,40.634773,-73.949721,45.0,786.0
606,BROOKLYN,3117318,134,ERASMUS STREET,321386932,NB,11226.0,2.0,YES,NaN,IN PROCESS,INITIAL,NB,1,05/11/2022,NaN,NaN,05/11/2022,PARTNERSHIP,N,05/12/2022 00:00:00,3906340,40.649718,-73.951018,40.0,824.0
658,BROOKLYN,3122029,150,TERRACE PLACE,340796151,NB,11218.0,1.0,YES,NaN,ISSUED,INITIAL,NB,1,05/10/2022,05/16/2022,10/26/2022,05/16/2022,INDIVIDUAL,N,05/17/2022 00:00:00,3905996,40.655274,-73.976642,39.0,50202.0
674,BROOKLYN,3815647,2922,FULTON ST,322039888,NB,11207.0,2.0,YES,NaN,ISSUED,INITIAL,NB,1,05/12/2022,05/12/2022,09/24/2022,05/12/2022,CORPORATION,N,05/13/2022 00:00:00,3906377,40.679221,-73.886894,37.0,1170.0


# `filing_status`
I want to look at initial filings.

In [230]:
permits['filing_status'].value_counts(dropna = False)

RENEWAL    135471
INITIAL     81936
Name: filing_status, dtype: int64

In [231]:
permits.shape

(217407, 26)

In [232]:
permits = permits[permits['filing_status'] == 'INITIAL']

In [233]:
permits.shape

(81936, 26)

In [234]:
permits.isnull().sum()

borough                     0
bin_no                      0
house_no                    0
street_name                 0
job_no                      0
job_type                    0
zip_code                   49
bldg_type                   2
residential                 0
work_type               81936
permit_status             439
filing_status               0
permit_type                 0
permit_sequence_no          0
filing_date                 0
issuance_date              66
expiration_date            53
job_start_date              0
owners_business_type      127
non-profit                  4
dobrundate                  0
permit_si_no                0
latitude                  328
longitude                 328
council_district          328
census_tract              328
dtype: int64

# Time
It appears the permits are issued really quickly. At least often. So that may not wind up being helpful. Might depend on location though.

# `bdlg_type`
I wanted to double check that this work on job_type and permit_type lines up with the bldg_type column, but when I examined it (using the backup to preserve the originals), the values were pretty nonsensical. Though I reached out to NYC Data to see if I could get an explanation, I decided to drop it at least until I hear back.

In [235]:
permits_backup['Bldg Type'].value_counts(dropna = False)

2.0    2908242
1.0     943559
NaN      57389
Name: Bldg Type, dtype: int64

It's odd that these are all 1, 2, or NaN because [the site](https://data.cityofnewyork.us/Housing-Development/DOB-Permit-Issuance/ipu4-2q9a) makes it seem like those only apply to number of families. But that doesn't line up with what I'm seeing here.

The DOB has a [list of building classifications](https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html), but it's much broader than this.

In [236]:
permits['bldg_type'].value_counts(dropna = False)

1.0    69135
2.0    12799
NaN        2
Name: bldg_type, dtype: int64

In [237]:
permits.drop(columns = 'bldg_type', inplace = True)

[This DOB webpage](https://www1.nyc.gov/site/buildings/property-or-business-owner/job-types-codes.page) provides permit type codes. EQ is chutes, cocoons, and construction hoists. PL is plumbing. SP is sprinkler. SD is standpipe. I don't need this data, so I'm going to drop this column.

# 'work_type'
This proved not to have any information that's of use.

In [238]:
permits['work_type'].value_counts(dropna = False)

NaN    81936
Name: work_type, dtype: int64

In [239]:
permits.drop(columns = 'work_type', inplace = True)

At this stage, I'm not certain if I'll use the `permit_status` but I'd like to be able to, so I'm going to drop the rows that have `NaN` for that column. They represent about .5% of my remaining data.

In [240]:
permits['permit_status'].value_counts(dropna = False)

ISSUED        79116
RE-ISSUED      2312
NaN             439
IN PROCESS       68
REVOKED           1
Name: permit_status, dtype: int64

In [241]:
permits['permit_status'].value_counts(dropna = False)[2]/permits.shape[0]

0.00535784026557313

In [242]:
permits = permits[permits['permit_status'].notnull()]

The `issuance_date` looks like it's got some formatting problems, but looking at the HUGE variety of dates, going back at least to 1990, this is something I'll want to explore in EDA. I'm dropping the `NaN`s in order to keep this column. It's going to eliminate about .3% of the current data.

In [243]:
permits.shape

(81497, 24)

In [244]:
permits['issuance_date'].value_counts(dropna = False)

05/24/2004     120
05/28/1997      86
04/04/2007      79
11/12/2003      78
04/07/1995      77
              ... 
2017-03-07       1
11/17/2020       1
2011-05-24       1
1997-01-28       1
07/17/2009       1
Name: issuance_date, Length: 9230, dtype: int64

In [245]:
permits['issuance_date'].value_counts(dropna = False)[0]/permits.shape[0]

0.0014724468385339338

In [246]:
permits = permits[permits['issuance_date'].notnull()]

In [247]:
permits.shape

(81431, 24)

The missing `owners_business_type` represents about .01% and I'll drop them because looking at that column might present some interesting findings.

In [248]:
permits['owners_business_type'].value_counts(dropna = False)

CORPORATION           36885
INDIVIDUAL            27622
PARTNERSHIP            9276
OTHER                  6864
HPD                     265
NYC AGENCY              180
NaN                     126
CONDO/CO-OP             125
OTHER GOV'T AGENCY       37
NYCHA/HHC                22
NYCHA                    20
NY STATE                  4
DOE                       2
HHC                       2
DCAS                      1
Name: owners_business_type, dtype: int64

In [249]:
permits['owners_business_type'].value_counts(dropna = False)[7]/permits.shape[0]

0.0015350419373457284

In [250]:
permits = permits[permits['owners_business_type'].notnull()]

In [251]:
permits.shape

(81305, 24)

In [252]:
permits.isnull().sum()

borough                   0
bin_no                    0
house_no                  0
street_name               0
job_no                    0
job_type                  0
zip_code                 49
residential               0
permit_status             0
filing_status             0
permit_type               0
permit_sequence_no        0
filing_date               0
issuance_date             0
expiration_date           0
job_start_date            0
owners_business_type      0
non-profit                0
dobrundate                0
permit_si_no              0
latitude                325
longitude               325
council_district        325
census_tract            325
dtype: int64

I found [this guide in stackoverflow](https://stackoverflow.com/a/69856510) to converting addresses to latitude/longitude. The answer below it is interesting too.

I used that answer in combination with [this article](https://towardsdatascience.com/geocode-with-python-161ec1e62b89) to write the code below to turn addresses into latitude and longitude data.

In [186]:
no_position = permits[permits['latitude'].isnull()]

In [187]:
no_position.head()

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,residential,permit_status,filing_status,permit_type,permit_sequence_no,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude,council_district,census_tract
3272,BROOKLYN,3429238,31,PORTE PLAZA,321647811,NB,11222.0,YES,ISSUED,INITIAL,NB,1,05/26/2022,05/26/2022,05/26/2023,05/26/2022,INDIVIDUAL,N,05/27/2022 00:00:00,3908658,NaN,NaN,NaN,NaN
4224,QUEENS,4620764,150-37,SULLIVAN DRIVE,421256492,NB,NaN,YES,ISSUED,INITIAL,NB,1,05/14/2019,05/14/2019,01/08/2020,05/14/2019,PARTNERSHIP,N,06/01/2022 00:00:00,2090592,NaN,NaN,NaN,NaN
30803,QUEENS,4052777,75-56,GRAND CENTRAL PKWY EXT,400091887,NB,11375.0,YES,ISSUED,INITIAL,NB,1,09/10/1992,09/10/1992,09/01/1993,09/10/1992,INDIVIDUAL,N,05/09/2022 00:00:00,39154,NaN,NaN,NaN,NaN
32229,STATEN ISLAND,5837036,47,GETZ AVENUE,500118990,NB,NaN,YES,ISSUED,INITIAL,NB,1,1994-06-28,1994-06-28,1994-09-23,1994-06-28,INDIVIDUAL,N,2017-11-03,449273,NaN,NaN,NaN,NaN
46221,QUEENS,4535342,10-23,BAY 32 STREET,400766293,NB,11691.0,YES,ISSUED,INITIAL,NB,1,1998-09-10,1998-09-10,1999-02-14,1998-09-10,CORPORATION,N,2017-11-03,817866,NaN,NaN,NaN,NaN


In [188]:
no_position.isnull().sum()

borough                   0
bin_no                    0
house_no                  0
street_name               0
job_no                    0
job_type                  0
zip_code                 45
residential               0
permit_status             0
filing_status             0
permit_type               0
permit_sequence_no        0
filing_date               0
issuance_date             0
expiration_date           0
job_start_date            0
owners_business_type      0
non-profit                0
dobrundate                0
permit_si_no              0
latitude                325
longitude               325
council_district        325
census_tract            325
dtype: int64

In [189]:
no_position = no_position[no_position['zip_code'].notnull()]

In [190]:
no_position.dtypes

borough                  object
bin_no                    int64
house_no                 object
street_name              object
job_no                    int64
job_type                 object
zip_code                float64
residential              object
permit_status            object
filing_status            object
permit_type              object
permit_sequence_no        int64
filing_date              object
issuance_date            object
expiration_date          object
job_start_date           object
owners_business_type     object
non-profit               object
dobrundate               object
permit_si_no              int64
latitude                float64
longitude               float64
council_district        float64
census_tract            float64
dtype: object

In [191]:
no_position['zip_code'].astype(int).astype(str)

3272       11222
30803      11375
46221      11691
51877      11691
61077      11239
           ...  
3824759    11415
3846027    11415
3846038    11415
3861929    10309
3884595    10306
Name: zip_code, Length: 280, dtype: object

In [192]:
no_position['address'] = no_position['house_no'].replace(" ", "") + " " + no_position['street_name'] + "," + "New York,NY," + no_position['zip_code'].astype(int).astype(str)



In [193]:
pd.set_option('display.max_colwidth', 100)

In [194]:
no_position['address'].head()

3272                   31 PORTE PLAZA,New York,NY,11222
30803    75-56 GRAND CENTRAL PKWY EXT,New York,NY,11375
46221             10-23 BAY 32 STREET,New York,NY,11691
51877             10-47 BAY 32 STREET,New York,NY,11691
61077                 702 EGAN STREET,New York,NY,11239
Name: address, dtype: object

In [196]:
geolocator = Nominatim(user_agent="myApp")

bad_addresses = []
for i in no_position.index:
    try:
        geocode = geolocator.geocode(no_position.loc[i, 'address'])
        no_position.loc[i, 'latitude'] = geocode.latitude
        no_position.loc[i, 'longitude'] = geocode.longitude
    except:
        bad_addresses.append(i)
        pass

In [197]:
len(no_position)

280

In [198]:
len(bad_addresses)

147

In [199]:
no_position.to_csv('../backup_data/backup_added_geolocate.csv')

In [200]:
no_position.isnull().sum()

borough                   0
bin_no                    0
house_no                  0
street_name               0
job_no                    0
job_type                  0
zip_code                  0
residential               0
permit_status             0
filing_status             0
permit_type               0
permit_sequence_no        0
filing_date               0
issuance_date             0
expiration_date           0
job_start_date            0
owners_business_type      0
non-profit                0
dobrundate                0
permit_si_no              0
latitude                147
longitude               147
council_district        280
census_tract            280
address                   0
dtype: int64

# Census Tract and Council District
ULTIMATELY DROPPING BECAUSE TOO INCONSISTENT AND NOT CLEAR IT WILL HELP, HAVING LONGITUDE AND LATITUDE (might be helpful in connecting demographic data... but I might need to just add that in later, using the `censusgeocode`)

This didn't help with council and census tracts. I looked at the [Census Gazetteer file](https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html) for NY, but it only provides one coordinate.

This [stackoverflow answer](https://stackoverflow.com/a/65120026) pointed me to `censusgeocode`.

This [NYC Data page](https://data.cityofnewyork.us/City-Government/2020-Census-Tracts-Mapped/weqx-t5xr) has 2020 Census Tracts.

Census tracts change over time, so I need to take that into account. Ultimately, if `censusgeocode` works, I might want to update all the census tracts.

In [203]:
no_position.shape

(280, 25)

In [204]:
no_position = no_position[no_position['latitude'].notnull()]

In [205]:
no_position.shape

(133, 25)

In [206]:
permits.shape

(81305, 24)

In [208]:
no_position.shape[0]/permits.shape[0]

0.0016358157554885924

In [209]:
permits.shape

(81305, 24)

In [253]:
permits = permits[permits['latitude'].notnull()]

In [254]:
permits.shape

(80980, 24)

OOPS MESSED THIS UP.

In [255]:
permits2 = permits.copy()

In [256]:
permits = pd.concat([permits, no_position])

In [257]:
permits.shape

(81113, 25)

In [259]:
permits.isnull().sum()

borough                     0
bin_no                      0
house_no                    0
street_name                 0
job_no                      0
job_type                    0
zip_code                    4
residential                 0
permit_status               0
filing_status               0
permit_type                 0
permit_sequence_no          0
filing_date                 0
issuance_date               0
expiration_date             0
job_start_date              0
owners_business_type        0
non-profit                  0
dobrundate                  0
permit_si_no                0
latitude                    0
longitude                   0
council_district          133
census_tract              133
address                 80980
dtype: int64

In [260]:
permits.drop(columns = ['address'], inplace = True)

In [261]:
permits.to_csv('../backup_data/permits_w_council_dist.csv')

For now, I'm just going to work off longitude and latitude, so I'll drop the council and census tracts. I'm going to set the zipcode NaNs to 99999 for now, since that's not a real zipcode. If I wind up wanting to use addresses, I'll deal with them as necessary

In [14]:
#permits = pd.read_csv('../backup_data/permits_w_council_dist.csv', index = False)

In [15]:
permits.drop(columns = ['council_district', 'census_tract'], inplace = True)

In [17]:
permits.set_index('Unnamed: 0', inplace = True)

In [21]:
permits.index.rename('', inplace = True)

In [22]:
permits[permits['zip_code'].isnull()]

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,residential,permit_status,filing_status,...,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude
,,,,,,,,,,,,,,,,,,,,,
8951,BROOKLYN,3428883,208,NOSTRAND AVENUE,321477709,NB,NaN,YES,ISSUED,INITIAL,...,06/21/2022,06/24/2022,03/07/2023,06/24/2022,CORPORATION,N,06/25/2022 00:00:00,3912506,40.693031,-73.952108
616634,QUEENS,4464044,9204,HOLLAND AVENUE,401273965,NB,NaN,YES,ISSUED,INITIAL,...,09/26/2001,09/26/2001,06/01/2002,09/26/2001,CORPORATION,N,05/09/2022 00:00:00,1465439,40.585362,-73.814802
2463150,QUEENS,4530251,1543,HASSOCK STREET,401841860,NB,NaN,YES,ISSUED,INITIAL,...,12/10/2004,12/10/2004,02/15/2005,12/10/2004,PARTNERSHIP,N,05/09/2022 00:00:00,316059,40.609080,-73.753366
3185810,BROOKLYN,3256842,506,BAY 44 STREET,300183414,NB,NaN,YES,ISSUED,INITIAL,...,12/16/1992,12/16/1992,07/30/1993,12/16/1992,CORPORATION,N,05/09/2022 00:00:00,382579,40.586631,-73.995824


In [24]:
permits.loc[permits['zip_code'].isnull(), 'zip_code'] = 99999

In [25]:
permits[permits['zip_code'].isnull()]

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,residential,permit_status,filing_status,...,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude
,,,,,,,,,,,,,,,,,,,,,


In [26]:
permits[permits['zip_code'] == 99999].head()

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,residential,permit_status,filing_status,...,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude
,,,,,,,,,,,,,,,,,,,,,
8951,BROOKLYN,3428883,208,NOSTRAND AVENUE,321477709,NB,99999.0,YES,ISSUED,INITIAL,...,06/21/2022,06/24/2022,03/07/2023,06/24/2022,CORPORATION,N,06/25/2022 00:00:00,3912506,40.693031,-73.952108
616634,QUEENS,4464044,9204,HOLLAND AVENUE,401273965,NB,99999.0,YES,ISSUED,INITIAL,...,09/26/2001,09/26/2001,06/01/2002,09/26/2001,CORPORATION,N,05/09/2022 00:00:00,1465439,40.585362,-73.814802
2463150,QUEENS,4530251,1543,HASSOCK STREET,401841860,NB,99999.0,YES,ISSUED,INITIAL,...,12/10/2004,12/10/2004,02/15/2005,12/10/2004,PARTNERSHIP,N,05/09/2022 00:00:00,316059,40.609080,-73.753366
3185810,BROOKLYN,3256842,506,BAY 44 STREET,300183414,NB,99999.0,YES,ISSUED,INITIAL,...,12/16/1992,12/16/1992,07/30/1993,12/16/1992,CORPORATION,N,05/09/2022 00:00:00,382579,40.586631,-73.995824


In [27]:
permits.isnull().sum()

borough                 0
bin_no                  0
house_no                0
street_name             0
job_no                  0
job_type                0
zip_code                0
residential             0
permit_status           0
filing_status           0
permit_type             0
permit_sequence_no      0
filing_date             0
issuance_date           0
expiration_date         0
job_start_date          0
owners_business_type    0
non-profit              0
dobrundate              0
permit_si_no            0
latitude                0
longitude               0
dtype: int64

In [28]:
permits.dtypes

borough                  object
bin_no                    int64
house_no                 object
street_name              object
job_no                    int64
job_type                 object
zip_code                float64
residential              object
permit_status            object
filing_status            object
permit_type              object
permit_sequence_no        int64
filing_date              object
issuance_date            object
expiration_date          object
job_start_date           object
owners_business_type     object
non-profit               object
dobrundate               object
permit_si_no              int64
latitude                float64
longitude               float64
dtype: object

In [30]:
permits.shape

(81113, 22)

# Taking Care of Duplicates
I opted to keep the oldest instance of the permit.

In [32]:
permits[permits.duplicated(subset = ['job_no'], keep = False)].shape

(5216, 22)

In [33]:
duplicated = permits[permits.duplicated(subset = ['job_no'], keep = False)]
duplicated.head()

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,residential,permit_status,filing_status,...,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude
,,,,,,,,,,,,,,,,,,,,,
3597,BROOKLYN,3428769,346,BERGEN STREET,301779396,NB,11217.0,YES,ISSUED,INITIAL,...,06/23/2008,06/23/2008,11/10/2008,06/23/2008,CORPORATION,N,06/26/2020 00:00:00,1832324,40.682824,-73.980487
12955,BRONX,2127279,710,EAST 221 ST,220492615,NB,10467.0,YES,ISSUED,INITIAL,...,07/17/2020,07/17/2020,09/27/2020,07/17/2020,CORPORATION,N,07/20/2020 00:00:00,3775974,40.884776,-73.861399
13936,QUEENS,4623155,10-06,127 STREET,421912210,NB,11356.0,YES,ISSUED,INITIAL,...,04/07/2022,04/08/2022,10/25/2022,04/08/2022,INDIVIDUAL,N,04/09/2022 00:00:00,3902390,40.788177,-73.841016
14039,QUEENS,4151229,75-23,166 STREET,421714363,NB,11366.0,YES,ISSUED,INITIAL,...,05/02/2019,05/02/2019,09/30/2020,05/02/2019,PARTNERSHIP,N,07/29/2020 00:00:00,3629269,40.726578,-73.802796
14554,BROOKLYN,3198213,2033,E 17 STREET,321791290,NB,11229.0,YES,ISSUED,INITIAL,...,01/23/2020,01/23/2020,09/30/2020,01/23/2020,INDIVIDUAL,N,07/29/2020 00:00:00,3732878,40.600443,-73.954771


In [37]:
pd.set_option('display.max_columns', 22)

In [40]:
duplicated['job_start_date'] = pd.to_datetime(duplicated['job_start_date'])

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_24394/1232839256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated['job_start_date'] = pd.to_datetime(duplicated['job_start_date'])


In [43]:
duplicated.sort_values(by='job_start_date', inplace = True)

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_24394/3323442690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated.sort_values(by='job_start_date', inplace = True)


In [44]:
duplicated.sort_values(by='job_no', inplace = True)

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_24394/2717639440.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated.sort_values(by='job_no', inplace = True)


In [46]:
duplicated.head()

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,residential,permit_status,filing_status,permit_type,permit_sequence_no,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude
,,,,,,,,,,,,,,,,,,,,,,
34435,MANHATTAN,1085345,2556A,8 AVENUE,100134070,NB,10030.0,YES,ISSUED,INITIAL,NB,1,1993-07-30,1993-07-30,1994-01-01,1993-07-30,OTHER,N,2017-11-03,346857,40.817374,-73.945898
3336873,MANHATTAN,1085345,2556A,8 AVENUE,100134070,NB,10030.0,YES,ISSUED,INITIAL,NB,1,07/30/1993,07/30/1993,01/01/1994,1993-07-30,OTHER,N,05/09/2022 00:00:00,454968,40.817374,-73.945898
16676,MANHATTAN,1085611,2572A,8 AVENUE,100134178,NB,10030.0,YES,ISSUED,INITIAL,NB,1,1992-12-29,1992-12-29,1994-01-01,1992-12-29,OTHER,N,2017-11-03,293450,40.817936,-73.945486
3221901,MANHATTAN,1085611,2572A,8 AVENUE,100134178,NB,10030.0,YES,ISSUED,INITIAL,NB,1,12/29/1992,12/29/1992,01/01/1994,1992-12-29,OTHER,N,05/09/2022 00:00:00,386074,40.817936,-73.945486
3411797,MANHATTAN,1084483,2543,8 AVENUE,100134258,NB,10030.0,YES,ISSUED,INITIAL,NB,1,01/28/1994,01/28/1994,01/01/1995,1994-01-28,OTHER,N,05/09/2022 00:00:00,528566,40.817275,-73.945999


In [48]:
duplicated.drop_duplicates(subset = 'job_no', inplace = True)

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_24394/1961017179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated.drop_duplicates(subset = 'job_no', inplace = True)


In [49]:
duplicated.shape

(2608, 22)

In [50]:
permits.shape

(81113, 22)

In [53]:
permits.drop_duplicates(subset = 'job_no', keep = False, inplace= True)

In [54]:
permits.shape

(75897, 22)

In [56]:
permits = pd.concat([permits, duplicated])

In [57]:
permits.shape

(78505, 22)

In [58]:
permits.columns

Index(['borough', 'bin_no', 'house_no', 'street_name', 'job_no', 'job_type',
       'zip_code', 'residential', 'permit_status', 'filing_status',
       'permit_type', 'permit_sequence_no', 'filing_date', 'issuance_date',
       'expiration_date', 'job_start_date', 'owners_business_type',
       'non-profit', 'dobrundate', 'permit_si_no', 'latitude', 'longitude'],
      dtype='object')

In [59]:
permits['filing_status'].value_counts()

INITIAL    78505
Name: filing_status, dtype: int64

In [60]:
permits['job_type'].value_counts()

NB    78505
Name: job_type, dtype: int64

In [61]:
permits['permit_type'].value_counts()

NB    78505
Name: permit_type, dtype: int64

I'm going to drop these because I either don't need them any more or can't determine what they are.

**Everything at this point is New Building (NB) for both job_type and permit_type. All residential, too**

In [62]:
permits.shape

(78505, 22)

In [63]:
permits.drop(columns = ['job_type', 'residential', 'filing_status','permit_type','permit_si_no', 'permit_sequence_no'], inplace = True)

In [68]:
permits.shape

(78505, 16)

make zip_code integer.
make filing_date, issuance_date, expiration_date, job_start_date into datetimes. Do I need expiration date?

what is permit_si_no? I can't find that anywhere online. Number for the permit? As opposed to the job?
permit_sequence_no?

can drop residential at this point because they're all residential. check it first to make sure.

Check the statuses to see what they look like

In [65]:
permits['zip_code'] = permits['zip_code'].astype(int)

In [69]:
permits['filing_date'] = pd.to_datetime(permits['filing_date'])

In [ ]:
permits['issuance_date'] = pd.to_datetime(permits['issuance_date'])

In [73]:
permits['expiration_date'] = pd.to_datetime(permits['expiration_date'])

In [75]:
permits['job_start_date'] = pd.to_datetime(permits['job_start_date'])

In [77]:
permits['dobrundate'] = pd.to_datetime(permits['dobrundate'])

In [78]:
permits.dtypes

borough                         object
bin_no                           int64
house_no                        object
street_name                     object
job_no                           int64
zip_code                         int64
permit_status                   object
filing_date             datetime64[ns]
issuance_date           datetime64[ns]
expiration_date         datetime64[ns]
job_start_date          datetime64[ns]
owners_business_type            object
non-profit                      object
dobrundate              datetime64[ns]
latitude                       float64
longitude                      float64
dtype: object

------

In [61]:
permits = pd.read_csv('data/nbrespermits.csv')

In [62]:
permits.dtypes

borough                  object
bin_no                    int64
house_no                 object
street_name              object
job_no                    int64
zip_code                  int64
permit_status            object
filing_date              object
issuance_date            object
expiration_date          object
job_start_date           object
owners_business_type     object
non-profit               object
dobrundate               object
latitude                float64
longitude               float64
dtype: object

In [63]:
permits['job_start_date'] = pd.to_datetime(permits['job_start_date'])

In [64]:
permits['job_start_date'].describe()

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_10379/1807671964.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  permits['job_start_date'].describe()


count                   78505
unique                   7718
top       2004-05-24 00:00:00
freq                      119
first     1989-09-08 00:00:00
last      2217-10-23 00:00:00
Name: job_start_date, dtype: object

# 5 Years Back
I eliminated data for jobs starting more than 5 years ago.

There were a handful of data that had problematic job start dates that were clearly from clerical errors. Most of those were from older permits than I'm interested and there were so few remaining I decided to eliminate those.

In [66]:
permits = permits[permits['job_start_date']> pd.to_datetime('2016-08-02')]

In [67]:
permits[permits['job_start_date']>pd.to_datetime('2022-09-30')].shape

(10, 16)

In [68]:
permits['expiration_date'] = pd.to_datetime(permits['expiration_date'])

In [69]:
permits.shape

(7384, 16)

In [71]:
permits[permits['expiration_date']<pd.to_datetime('2016-08-02')]

,borough,bin_no,house_no,street_name,job_no,zip_code,permit_status,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,latitude,longitude
21460,BROOKLYN,3387710,168,CLYMER STREET,301003027,11211,ISSUED,2002-10-17,2002-10-17,2003-08-14,2020-10-17,INDIVIDUAL,N,2022-05-09,40.706954,-73.963001
34263,QUEENS,4102779,151-38,7 AVENUE,402290588,11357,ISSUED,2006-08-04,2006-08-10,2007-03-07,2206-08-10,INDIVIDUAL,N,2022-05-09,40.795141,-73.812218
45687,BRONX,2128376,943,FTELEY AVE,220361935,10473,ISSUED,2014-09-02,2014-09-02,2015-06-03,2104-09-02,INDIVIDUAL,N,2022-05-09,40.823810,-73.870284
48363,BROOKLYN,3413793,1269,EAST 54TH STREET,320593443,11234,RE-ISSUED,2015-11-04,2015-11-13,2016-06-30,2016-11-18,CORPORATION,N,2022-05-09,40.626969,-73.923825
48736,BRONX,2102544,2082,HONEYWELL AVENUE,200614193,10460,ISSUED,2000-11-20,2000-11-20,2001-02-26,2200-11-20,PARTNERSHIP,N,2022-05-09,40.844549,-73.882785
71441,BROOKLYN,3388038,772,HERKIMER STREET,301467240,11233,ISSUED,2003-01-17,2003-01-17,2003-11-15,2030-01-17,CORPORATION,N,2022-05-09,40.678436,-73.927880


In [72]:
permits = permits[permits['expiration_date']>pd.to_datetime('2016-08-02')]

In [73]:
permits.shape

(7378, 16)

In [74]:
permits = permits[permits['job_start_date']<pd.to_datetime('2022-09-30')]

In [75]:
permits.shape

(7372, 16)

I wanted to screen out other possible transcription error. For example, I saw a job_start_date in 2020 that was almost certainly a transcription error for 2002. For this reason, I looked at the time between job_start_date and expiration. I can't find information online about how long before permits expire, typically, but none of the periods seem unreasonable, as they all fall within 2 years of the job_start_date, so I'm not going to change anything based on this.

[This stackoverflow answer](https://stackoverflow.com/a/37588991) helped me understand how to use .days to get the number of days.

In [76]:
permits['issuance_date'] = pd.to_datetime(permits['issuance_date'])

In [78]:
permits['difference'] = permits['expiration_date']-permits['job_start_date']
permits['difference'] = permits['difference'].map(lambda x: x.days)
permits['difference'].head()

1    365
2    365
3    365
4    288
5    344
Name: difference, dtype: int64

In [81]:
permits[permits['difference'] > 550]

,borough,bin_no,house_no,street_name,job_no,zip_code,permit_status,filing_date,issuance_date,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,latitude,longitude,difference
37123,BROOKLYN,3171499,1169,EAST 9 STREET,321770810,11230,ISSUED,2019-05-24,2019-05-28,2021-01-31,2019-05-28,INDIVIDUAL,N,2022-05-09,40.620648,-73.966345,614
42172,QUEENS,4201081,101-64,132 STREET,421415686,11419,ISSUED,2019-04-11,2019-04-11,2020-10-31,2019-04-11,NYCHA/HHC,N,2022-05-09,40.691674,-73.814738,569
42557,QUEENS,4026146,32-11,60 STREET,421717100,11377,ISSUED,2019-05-17,2019-05-22,2020-11-30,2019-05-22,INDIVIDUAL,N,2022-05-09,40.755452,-73.902408,558
42561,QUEENS,4548904,32-15,60 STREET,421717155,11377,ISSUED,2019-05-17,2019-05-22,2020-11-30,2019-05-22,INDIVIDUAL,N,2022-05-09,40.755391,-73.902397,558
43579,QUEENS,4619476,31-17,37TH ST,420664597,11103,ISSUED,2019-05-08,2019-05-08,2020-11-30,2019-05-08,CORPORATION,N,2022-05-09,40.761448,-73.919181,572
44503,BRONX,2129024,957,TILDEN ST,210177921,10469,ISSUED,2019-03-27,2019-03-27,2020-09-30,2019-03-27,INDIVIDUAL,N,2022-05-09,40.876821,-73.858059,553
47043,QUEENS,4258336,147-48,FERNDALE AVE,421713596,11435,ISSUED,2019-05-22,2019-05-22,2020-11-30,2019-05-22,CORPORATION,N,2022-05-09,40.690615,-73.798673,558
52797,BROOKLYN,3172842,1466,EAST 5TH ST,321636002,11230,ISSUED,2019-04-12,2019-04-12,2020-10-31,2019-04-12,INDIVIDUAL,N,2022-05-09,40.615182,-73.970177,568
53789,BROOKLYN,3426084,1,CROSBY AVENUE,320912599,11207,ISSUED,2019-06-12,2019-06-12,2020-12-31,2019-06-12,CORPORATION,N,2022-05-09,40.681681,-73.896051,568
54821,BROOKLYN,3131942,2042,59 STREET,321926751,11204,ISSUED,2019-06-27,2019-06-27,2020-12-31,2019-06-27,CORPORATION,N,2022-05-09,40.619318,-73.981229,553


In [82]:
permits.shape

(7372, 17)

I just want to look at the `'ISSUED'` columns, just to be sure the focus is on buildings that were first started in this period.

In [84]:
permits['permit_status'].value_counts()

ISSUED        7176
RE-ISSUED      195
IN PROCESS       1
Name: permit_status, dtype: int64

Going to drop a few more columns.

In [85]:
permits = permits[permits['permit_status'] == 'ISSUED']

In [86]:
permits['permit_status'].value_counts()

ISSUED    7176
Name: permit_status, dtype: int64

In [83]:
permits.columns

Index(['borough', 'bin_no', 'house_no', 'street_name', 'job_no', 'zip_code',
       'permit_status', 'filing_date', 'issuance_date', 'expiration_date',
       'job_start_date', 'owners_business_type', 'non-profit', 'dobrundate',
       'latitude', 'longitude', 'difference'],
      dtype='object')

In [87]:
permits = permits[['borough', 'bin_no', 'house_no', 'street_name', 'job_no', 'zip_code',
       'job_start_date', 'owners_business_type', 'non-profit',
       'latitude', 'longitude']]

In [89]:
permits.shape

(7176, 11)

In [90]:
permits.to_csv('data/nbrespermits.csv', index = False)

-----

# Saving For Now
I have some work here that helps identify census tract, I'm going to save it for the moment in case I decide to use it later.

In [164]:
pd.set_option('display.max_columns', None)

In [148]:
result = cg.coordinates(x=no_position.loc[651]['longitude'], y=no_position.loc[651]['latitude'])

In [156]:
int(result['Census Tracts'][0]['TRACT'])

74500

I can use the above to get census tracts

In [157]:
permits[permits['census_tract'] == 74500]

,borough,bin_no,house_no,street_name,job_no,job_type,zip_code,residential,permit_status,filing_status,...,expiration_date,job_start_date,owners_business_type,non-profit,dobrundate,permit_si_no,latitude,longitude,council_district,census_tract
